---
title: "Function Calling and Structured Outputs in LLMs with LangChain and OpenAI"
date: 2025-06-30
description-meta: "Understand how to use function calling and structured outputs in LLMs with LangChain and OpenAI"
categories:
  - llm
  - function-calling
  - structured-outputs
  - openai
---

## Function calling

Function calling or tool use refers to the ability to get LLMs to invoke external tools or functions. This works by giving the LLM access to a set of predefined tools that it can invoke at any point.

It's relevant because it gives LLMs more capabilities, allows them to integrate with systems, and enables complex task automation. This was one of the key features that unlocked agents.

The usual flow is:
1. The user asks a question
2. The LLM decides if it needs to use a tool
3. If it does, it invokes the tool and gets the output from the tool
4. The LLM then uses the output to answer the user's question

Here's a diagram that illustrates how function calling works:

![Function calling flow](./images/function-calling-structured-outputs/diagram.png)

AI developers are increasingly using function calling to build more complex systems. You can use it to:

- Get information from a CRM, DB, etc
- Perform calculations (e.g., generate an estimae for a variable, financial calculations)
- Manipulate data (e.g., data cleaning, data transformation)
- Interact with external systems (e.g., booking a flight, sending an email)

## Structured outputs

Structured outputs are a group of methods that “ensure that model outputs adhere to a specific structure”^[["We Need Structured Output": Towards User-centered Constraints on LLM Output. MX Liu et al. 2024](https://arxiv.org/abs/2404.07362)]. With proprietary models, this usually means a JSON schema. Most propietary providers give you different methods to define a JSON schema.

In open-weight models, a structure can mean anything from a JSON schema to a specific regex pattern. You can use [outlines](https://dottxt-ai.github.io/outlines/latest/) for this.

Structured outputs are very useful to create agentic systems, as they greatly simplify the communication between components. It's a lot easier to parse the output of a JSON object than a free-form text. But as with any other component on your system, you should measure its impact. It might [impact the performance](https://dylancastillo.co/posts/say-what-you-mean-sometimes.html) of your task.

In the next sections, I'll show you how to use function calling and structured outputs with OpenAI.

## Prerequisites

To follow this tutorial you'll need to:

1. Sign up and generate an API key in [OpenAI](https://platform.openai.com/docs/overview).
2. Set the API key as an environment variable called `OPENAI_API_KEY`.
3. Create a virtual environment in Python and install the requirements:

```bash
python -m venv venv
source venv/bin/activate
pip install langchain langchain-openai python-dotenv jupyter
```

Once you've completed the steps above, you can run copy and paste the code from the next sections. You can also download the notebook from [here](function-calling-structured-outputs.ipynb).

In [4]:
#| output: false
import os
from typing import Literal

import requests
from dotenv import load_dotenv
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langsmith import traceable
from pydantic import BaseModel, Field

load_dotenv()

True

## Single tool

In [ ]:
model = ChatOpenAI(model_name="gpt-4.1-mini")

In [ ]:
@tool
def find_weather(latitude: float, longitude: float):
    """Get the weather of a given latitude and longitude"""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]["temperature_2m"]


tools_mapping = {
    "find_weather": find_weather,
}

model_with_tools = model.bind_tools([find_weather])


@traceable
def get_response(question: str):
    messages = [
        SystemMessage(
            "You're a helpful assistant. Use the tools provided when relevant."
        ),
        HumanMessage(question),
    ]
    ai_message = model_with_tools.invoke(messages)
    messages.append(ai_message)

    for tool_call in ai_message.tool_calls:
        selected_tool = tools_mapping[tool_call["name"]]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)

    ai_message = model_with_tools.invoke(messages)
    messages.append(ai_message)

    return ai_message.content


response = get_response("What's the weather in Tokyo?")
print(response)

## Multiple tools

In [2]:
model = ChatOpenAI(model_name="gpt-4.1-mini")


@tool
def get_weather(latitude: float, longitude: float):
    """Get the weather of a given latitude and longitude"""
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data["current"]["temperature_2m"]


@tool
def check_guidelines(drafted_response: str) -> str:
    """Check if a given response follows the company guidelines"""
    model = ChatOpenAI(model_name="gpt-4.1-mini")
    response = model.invoke(
        [
            SystemMessage(
                "You're a helpful assistant. Your task is to check if a given response follows the company guidelines. The company guidelines are that responses should be written in the style of a haiku. You should reply with 'OK' or 'REQUIRES FIXING' and a short explanation."
            ),
            HumanMessage(f"Current response: {drafted_response}"),
        ]
    )
    return response.content


tools_mapping = {
    "get_weather": get_weather,
    "check_guidelines": check_guidelines,
}

model_with_tools = model.bind_tools([get_weather, check_guidelines])


@traceable
def get_response(question: str):
    messages = [
        SystemMessage(
            "You're a helpful assistant. Use the tools provided when relevant. Then draft a response and check if it follows the company guidelines. Only respond to the user after you've validated and modified the response if needed."
        ),
        HumanMessage(question),
    ]
    ai_message = model_with_tools.invoke(messages)
    messages.append(ai_message)

    while ai_message.tool_calls:
        for tool_call in ai_message.tool_calls:
            selected_tool = tools_mapping[tool_call["name"]]
            tool_msg = selected_tool.invoke(tool_call)
            messages.append(tool_msg)
        ai_message = model_with_tools.invoke(messages)
        messages.append(ai_message)

    return ai_message.content


response = get_response("What is the temperature in Madrid?")
print(response)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

## Structured outputs

In [ ]:
class DocumentInfo(BaseModel):
    category: Literal["financial", "legal", "marketing", "pets", "other"]
    summary: str


model = ChatOpenAI(model="gpt-4.1-mini", temperature=0)


def get_document_info(document: str) -> DocumentInfo:
    model_with_structure = model.with_structured_output(DocumentInfo)
    response = model_with_structure.invoke(document)
    return response


document_text = dedent("""
This is a document about cats. Very important document. It explain how cats will take over the world in 20230.
"""
)
document_info = get_document_info("I'm a document about a cat")
print(document_info)

## Conclusion